In [13]:
# Import Dataset and useful libraries 
import os 
import pandas as pd 
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from datetime import datetime
os.chdir('../scripts/')
import utils as util 

source1 = "../../Data/sample_submission.csv"
source2 = "../../Data/store.csv"
source3 = "../../Data/test.csv"
source4 = "../../Data/train.csv"

sample_submission = util.read_csv_file(source1).get('data')
store = util.read_csv_file(source2).get('data')
test = util.read_csv_file(source3).get('data')
train = util.read_csv_file(source4).get('data')

c:\Users\Administrator\Documents\Projects\Tech company\TenX\projects\Rossmann_Pharmaceuticals_ML\src\scripts\utils.py:5: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path)


In [14]:
store.info()
store.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1115 entries, 0 to 1114
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Store                      1115 non-null   int64  
 1   StoreType                  1115 non-null   object 
 2   Assortment                 1115 non-null   object 
 3   CompetitionDistance        1112 non-null   float64
 4   CompetitionOpenSinceMonth  761 non-null    float64
 5   CompetitionOpenSinceYear   761 non-null    float64
 6   Promo2                     1115 non-null   int64  
 7   Promo2SinceWeek            571 non-null    float64
 8   Promo2SinceYear            571 non-null    float64
 9   PromoInterval              571 non-null    object 
dtypes: float64(5), int64(2), object(3)
memory usage: 87.2+ KB


,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


In [15]:
test.info()
test.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41088 entries, 0 to 41087
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             41088 non-null  int64  
 1   Store          41088 non-null  int64  
 2   DayOfWeek      41088 non-null  int64  
 3   Date           41088 non-null  object 
 4   Open           41077 non-null  float64
 5   Promo          41088 non-null  int64  
 6   StateHoliday   41088 non-null  object 
 7   SchoolHoliday  41088 non-null  int64  
dtypes: float64(1), int64(5), object(2)
memory usage: 2.5+ MB


,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.0,1,0,0
1,2,3,4,2015-09-17,1.0,1,0,0
2,3,7,4,2015-09-17,1.0,1,0,0
3,4,8,4,2015-09-17,1.0,1,0,0
4,5,9,4,2015-09-17,1.0,1,0,0


In [16]:
train.info()
train.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 9 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   Store          1017209 non-null  int64 
 1   DayOfWeek      1017209 non-null  int64 
 2   Date           1017209 non-null  object
 3   Sales          1017209 non-null  int64 
 4   Customers      1017209 non-null  int64 
 5   Open           1017209 non-null  int64 
 6   Promo          1017209 non-null  int64 
 7   StateHoliday   1017209 non-null  object
 8   SchoolHoliday  1017209 non-null  int64 
dtypes: int64(7), object(2)
memory usage: 69.8+ MB


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


## Preprocssing the Data
It is important to process the data into a format where it can be fed to a machine learning model. This typically means converting all non-numeric columns to numeric, handling NaN values, and generating new features from already existing features. 
#
In our case, you have a few datetime columns to preprocess. you can extract the following from them:
#
weekdays,
weekends ,
number of days to holidays,
Number of days after a holiday,
Beginning of the month, mid-month, and end of the month
(think of more features to extract), extra marks for it
As a final thing, you have to scale the data. This helps with predictions especially when using machine learning algorithms that use Euclidean distances. you can use the standard scaler in sklearn for this.

## 1. Fill missing values in Store dataset 
- For competition Distance I will try to fill with the median value 
- For others I will try to fill with zero 
- For Promo Interval, fill unknown

In [17]:
store['CompetitionDistance'] = store['CompetitionDistance'].fillna(store['CompetitionDistance'].median())
store['CompetitionOpenSinceMonth'] = store['CompetitionOpenSinceMonth'].fillna(0)
store['CompetitionOpenSinceYear'] = store['CompetitionOpenSinceYear'].fillna(0)
store['Promo2SinceWeek'] = store['Promo2SinceWeek'].fillna(0)
store['Promo2SinceYear'] = store['Promo2SinceYear'].fillna(0)
store['PromoInterval'] = store['PromoInterval'].fillna("Unknown")

# after that check any missing values in the dataset 
print(store.isnull().sum())

Store                        0
StoreType                    0
Assortment                   0
CompetitionDistance          0
CompetitionOpenSinceMonth    0
CompetitionOpenSinceYear     0
Promo2                       0
Promo2SinceWeek              0
Promo2SinceYear              0
PromoInterval                0
dtype: int64


### For the following reason I will merge the train and test data in the store 
**Incorporate Store-Specific feature** - The `store` dataset contains detailed attribute about each store (e.g `storeType`,`Assortment`, `competitionDestance`,etc.) that are not present in the train or test dataset. 
- **Add predictive power to the model** 
- **Ensure consistency Between Datasets** 
- **Prepare the Dataset for Modeling** 


In [18]:
# Merge train/test with store
train = train.merge(store, on='Store', how='left')
test = test.merge(store, on='Store', how='left')

### For the following reason I will change `Date` column to a datetime format. 
- Enable feature extraction - the datetime format allows you to extract meaningful time-based featurs, such as year,month,dayofwork .. 
- Time-based calculations - Datetime format enables easy time-based calculations, such as Days of holidays, Durations
- Handle Time series data
- Efficient Filtering and Querying 
- Simplify Data Visualization
- Prevent Errors 

In [19]:
# Convert 'Date' to datetime
train['Date'] = pd.to_datetime(train['Date'])
test['Date'] = pd.to_datetime(test['Date'])

The provided code extracts date-related features from the `Date` column in both the train and test datasets. These features are crucial for capturing patterns and trends related to time, which can significantly improve the performance of machine learning models. 

In [20]:
# Extract date features
for df in [train, test]:
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month
    df['WeekOfYear'] = df['Date'].dt.isocalendar().week
    df['DayOfYear'] = df['Date'].dt.dayofyear
    df['Day'] = df['Date'].dt.day
    df['IsWeekend'] = df['DayOfWeek'].apply(lambda x: 1 if x in [6, 7] else 0)
    df['IsBeginningOfMonth'] = df['Day'].apply(lambda x: 1 if x <= 10 else 0)
    df['IsMidMonth'] = df['Day'].apply(lambda x: 1 if 11 <= x <= 20 else 0)
    df['IsEndOfMonth'] = df['Day'].apply(lambda x: 1 if x > 20 else 0)

### One-hot encoding for categorical columns 
This is a critical preprocessing step in machine learning for models that requrie numerical inputs. Its purpose is as follow;
- Machine learning algorithms (especially traditional ones like linear regression, decision trees, etc.) cannot directly process categorical data because they expect numeric inputs.
- When you leave categorical data as strings or assign them numeric labels (e.g., a=1, b=2), the model might incorrectly assume an ordinal relationship (e.g., 2 > 1 for b and a). One-hot encoding prevents this by treating each category as an independent feature.

In [21]:
# One-hot encoding for categorical columns
categorical_cols = ['StoreType', 'Assortment', 'StateHoliday', 'PromoInterval']
train = pd.get_dummies(train, columns=categorical_cols, drop_first=True)
test = pd.get_dummies(test, columns=categorical_cols, drop_first=True)

### Handle Scaling 
Purpose of scaling 
- Normalize feature magnitudes since scaling ensures all features have comparable magnitudes 
- Improve Model Convergence: Gradient-based models (e.g., logistic regression, neural networks) converge faster during training when features are scaled.
- Prevent Bias in Models: Unscaled features can lead to biased predictions, as the model may give undue importance to high-magnitude features.

In [23]:
# Handle scaling
scaler = StandardScaler()
numerical_cols = ['CompetitionDistance', 'Promo2SinceWeek', 'Promo2SinceYear']
train[numerical_cols] = scaler.fit_transform(train[numerical_cols])
test[numerical_cols] = scaler.transform(test[numerical_cols])

In [25]:
# Finally Save preprocessed data
train.to_csv("../../Data/train_preprocessed.csv", index=False)
test.to_csv("../../Data/test_preprocessed.csv", index=False)

## summary 
---
### 1. Missing Data Handling
Filled missing values in the store dataset:
#
-CompetitionDistance: Filled with the median value.
-CompetitionOpenSinceMonth and CompetitionOpenSinceYear: Filled with 0 (indicating no competition).
-Promo2SinceWeek, Promo2SinceYear: Filled with 0 (indicating no promotion).
-PromoInterval: Filled with "Unknown" for missing entries.
### 2. Dataset Merging
-Merged store dataset with both train and test datasets using the Store column. This integrates additional features about stores (e.g., competition, promotions) into the training and testing data.
### 3. Datetime Feature Extraction
#
- Converted the Date column in both train and test datasets to datetime format.
- Extracted additional features from the Date column:
- Year, Month, WeekOfYear, DayOfYear, Day.
- Flags for specific time periods:
- IsWeekend (Saturday and Sunday).
- IsBeginningOfMonth, IsMidMonth, IsEndOfMonth.
### 4. Encoding Categorical Features
#
- Performed one-hot encoding on categorical columns (StoreType, Assortment, StateHoliday, PromoInterval) in both train and test datasets. This converts categories into binary numeric columns for machine learning models.
### 5. Scaling Numerical Features
#
- Scaled numerical columns in both datasets using StandardScaler to standardize the data (mean = 0, std = 1):
- Columns in both train and test: CompetitionDistance, Promo2SinceWeek, Promo2SinceYear.
- Columns exclusive to train: Sales (target variable) and Customers.